# Monte Carlo Simulation

Here we will perform a Monte Carlo simulation to investigate the finite sample coverage of traditional and robust standard errors. The simulation is similar to one found in Angrist & Pischke (MHE, Chapter 8).

In [1]:
# load packages
import numpy as np
import statsmodels.api as sm
import pandas as pd

In [2]:
# write simulation function
def sim_fun(n=30, r=0.1, s2=0.4, b0=0.5, b1=1):
    # simulate data
    n1 = int(np.floor(n*r)) # number of treated
    d = np.concatenate([np.ones(n1),np.zeros(n-n1)]) # create n1 treated and n-n1 non treated
    y = b0 + b1*d + np.random.normal(0, np.sqrt((1-d)*1 + d*s2),n) # outcome
    
    # fit ols
    D = sm.add_constant(d)
    ols = sm.OLS(y, D).fit()
    
    # compute traditional ci
    ci = ols.conf_int()
    
    # check if trad ci covers true b1
    trad_in = (b1 >= ci[1][0]) and (b1 <= ci[1][1])
    
    # compute robust ci
    ols_robust=sm.OLS(y, D).fit(cov_type = "HC0")
    rob_ci=ols_robust.conf_int()
    
    
    # check if rob ci covers true b1
    rob_in = (b1 >= rob_ci[1][0]) and (b1 <= rob_ci[1][1])
    return (trad_in,rob_in)

In [3]:
# construct simulation scenarios
n  = [30, 100, 500]
s2 = [1, 5, 10]

# create dataframe to store results
sims = pd.DataFrame( {'n': n*3, 's2': np.repeat(s2,3),'trad': np.zeros(9),'rob':np.zeros(9)})

In [4]:
# run simulations
# 1000 replications per scenario

for i in range(len(sims)):
    print("Simulation", i+1, "of", len(sims), "\n")
    print("-params: n =", sims.iloc[i]['n'], ",", "s2 =", sims.iloc[i]['s2'], "\n")
  
    # simulates 1000 times
    sims_i=np.zeros((1000,2))
    for j in range(1000):
        result = sim_fun(n=int(sims.iloc[i]['n']),s2=sims.iloc[i]['s2'])
        sims_i[j,0] = result[0]
        sims_i[j,1] = result[1]
    # computes coverage
    sims.at[i,'trad'] = np.mean(sims_i[:,0])
    sims.at[i,'rob'] = np.mean(sims_i[:,1])

Simulation 1 of 9 

-params: n = 30.0 , s2 = 1.0 

Simulation 2 of 9 

-params: n = 100.0 , s2 = 1.0 

Simulation 3 of 9 

-params: n = 500.0 , s2 = 1.0 

Simulation 4 of 9 

-params: n = 30.0 , s2 = 5.0 

Simulation 5 of 9 

-params: n = 100.0 , s2 = 5.0 

Simulation 6 of 9 

-params: n = 500.0 , s2 = 5.0 

Simulation 7 of 9 

-params: n = 30.0 , s2 = 10.0 

Simulation 8 of 9 

-params: n = 100.0 , s2 = 10.0 

Simulation 9 of 9 

-params: n = 500.0 , s2 = 10.0 



In [5]:
sims

,n,s2,trad,rob
0,30,1,0.954,0.785
1,100,1,0.957,0.918
2,500,1,0.958,0.959
3,30,5,0.693,0.752
4,100,5,0.732,0.917
5,500,5,0.727,0.930
6,30,10,0.596,0.757
7,100,10,0.633,0.908
8,500,10,0.631,0.951
